# Workflow for e-chemistry 
Version (10.07.2023) (python version 3.10.6)

This workspace is dedicated to e-chemistry, providing analysis tools for CP, CA, and CV. Please note that these tools are specifically designed to work with a particular file format.

The CP class is developed and maintained by the Elias Klemm group at the University of Stuttgart. For CA and CV analysis, you can utilize the projects from the Kristina Tschulik group at the University of Bochum and the Biprajit Sarkar group at the University of Stuttgart.

# Packages for the Analysis tools

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import copy
from scipy.optimize import curve_fit
from sdRDM import DataModel
from tabulate import tabulate
from scipy.integrate import trapz
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import json
################################################################
#imports the Analysis tools from the subfolder functions. They are in a Py file called Analysis
from tools.analysis_tools import ReferenceCalculator,Chronopotentiometry,Chronoamperometry,CyclicVoltammetry
%load_ext autoreload
%autoreload 2

# Creation of the subfolders data and plots
By exicuting the following cell two subfolder were created called data and plots. In the data folder you can store your measurement data and the plot folder is the folder were your plots get created. The plot folder is neccasary if you want to save your plots.

In [ ]:
path_program=os.getcwd()
data="/data"
plots="/plots"

path_data= path_program + data
path_plots=path_program+ plots
if not os.path.exists(path_data):
    os.makedirs(path_data)
if not os.path.exists(path_plots):
    os.makedirs(path_plots)


# Datamodel from GitHub
The following cell loads the Datamodel from GitHub and stores it in the variable lib. The Datamodel is stored in e_chem.

In [32]:

lib = DataModel.from_markdown("specifications/Electrochemistry.md")
#lib=DataModel.from_git(url="https://github.com/FAIRChemistry/datamodel-electrochemistry.git")
e_chem=lib.Dataset()



In [5]:
json_file_path = "e_chem.json"

# Die Daten in JSON konvertieren und in die Datei schreiben
with open(json_file_path, "w") as json_file:
    json.dump(e_chem.__dict__, json_file)

TypeError: Object of type Experiment is not JSON serializable

In [ ]:
json_file_path = "e_chem.json"

# Daten aus der JSON-Datei lesen
with open(json_file_path, "r") as json_file:
    data_dict = json.load(json_file)

# Die Daten wieder in ein lib.Dataset()-Objekt konvertieren
e_chem = lib.Dataset()
e_chem.__dict__.update(data_dict)

# Data formats
## Chronopotentiometry
The Chronopotentiometry class works with the formats: 
- .DTA 
## Chronoamperometry 
The Chronoamperometry class accepts the formats:
- .csv (group Tschulik)
- .dat (group Tschulik)
- .txt (group Sarkar)
### Cyclic Voltammetry 
The CyclicVoltammetry class accepts the formats:
- .csv (group Tschulik & Sarkar)
- .xlsx (group Sarkar)
  
Please note that the CSV file needs to be saved in UTF-8 format. To ensure this, open the CSV file in an editor like Excel and save it as a UTF-8 CSV file. This step is necessary to properly handle special characters and encoding. However, this encoding consideration is not required for XLSX files.

Unlike CSV files, XLSX files do not require specific encoding considerations. They inherently support a wider range of characters, including special characters, without the need for explicit encoding settings. Additionally, the data parser for XLSX files often works faster due to its utilization of the "C" engine.

# Input in the data model
The following shows how to store your data in the datamodel. If you want to see which variable are in an object of the model you can use the `.__dict__` command. 
For example, if you want to see which object the electrode_setup was executed on use:

`lib.ElectrodeSetup().__dict__`

In [4]:
lib.ElectrodeSetup().__dict__

{'id': 'electrodesetup0',
 'working_electrode': None,
 'working_electrode_area': None,
 'working_electrode_area_unit': None,
 'counter_electrode': None,
 'reference_electrode': None,
 'reference_electrode_salt': None,
 'reference_electrode_salt_concentration': None,
 'reference_electrode_salt_concentration_unit': None}

If the filename object in lib.Experiments requires the path along with the filename, you should provide the path accordingly. Here are the guidelines:

If your experiment files are located in the same directory as your notebook, you can simply use the filename without any path.
Example: filename = "experiment.csv"

If you store your experiment files in a subfolder named "data", you should include the path "data/" along with the filename.
- Example: filename = "data/experiment.csv"

Make sure to adjust the path and filename according to the actual location of your experiment files

In [33]:
"""The following shows ho to store experiments into the list of experiments"""
experiment0=lib.Experiment(name="Cu 12 nm",filename="data/Cu12nm.DTA",type="CP")
experiment1=lib.Experiment(name="Cu 14 nm",filename="data/Cu14nm.DTA",type="CP")
experiment2=lib.Experiment(name="CA1",filename="data/CA1.dat",type="CA")
experiment3=lib.Experiment(name="CA",filename="data/CA_Sarkar.txt",type="CA")
experiment4=lib.Experiment(name="CV",filename="data/SarkarCV_utf8.csv",type="CV")
experiment5=lib.Experiment(name="CV",filename="data/CV_all2.xlsx",type="CV")
experiment6=lib.Experiment(name="CV",filename="data/CV_all.xlsx",type="CV")
experiment7=lib.Experiment(name="Re Complex",filename="data/Ox_100mVs.csv",type="CV")
experiment8=lib.Experiment(name="Fc",filename="data/Ref_Fc_100mVs.csv",type="CV")
e_chem.experiments=[experiment0,experiment1,experiment2,experiment3,experiment4,experiment5,experiment6,experiment7,experiment8]
#### CP
e_chem.experiments[0].electrode_setup=lib.ElectrodeSetup(counter_electrode="Pt",reference_electrode="Hg/HgO",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M",working_electrode="Au",working_electrode_area=1,working_electrode_area_unit="cm^2")
e_chem.experiments[1].electrode_setup=lib.ElectrodeSetup(counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M",working_electrode="Au",working_electrode_area=1,working_electrode_area_unit="cm^2")
e_chem.experiments[0].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit=lib.enums.ConcentrationUnits.MOLAR,pH=10)
e_chem.experiments[1].electrolyte=lib.Electrolyte(solvent="MeCN",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit=lib.enums.ConcentrationUnits.MOLAR,pH=10)
e_chem.experiments[0].analysis=lib.Analysis(cp=lib.CP(induced_current=[200],induced_current_unit="mA",measurement_potential_unit="V",measurement_time_unit="s"))
e_chem.experiments[1].analysis=lib.Analysis(cp=lib.CP(induced_current=[200],induced_current_unit="mA",measurement_potential_unit="V",measurement_time_unit="s"))
#### CA
e_chem.experiments[2].electrode_setup=lib.ElectrodeSetup(working_electrode_area=1,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Hg/HgO",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[3].electrode_setup=lib.ElectrodeSetup(working_electrode_area=1,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[2].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[3].electrolyte=lib.Electrolyte(solvent="MeCN",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[2].analysis=lib.Analysis(ca=lib.CA(induced_potential=[20],induced_potential_unit="V",measurement_current_unit="A",measurement_time_unit="s"))
e_chem.experiments[3].analysis=lib.Analysis(ca=lib.CA(induced_potential=[20],induced_potential_unit="V",measurement_current_unit="A",measurement_time_unit="s"))
#### CV
e_chem.experiments[4].electrode_setup=lib.ElectrodeSetup(working_electrode="Au",working_electrode_area=2,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[4].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[4].analysis=lib.Analysis(cv=lib.CV(scan_rate=20,scan_rate_unit="mV/s",measurement_current_unit="µA",measurement_potential_unit="V"))
e_chem.experiments[5].electrode_setup=lib.ElectrodeSetup(working_electrode="Au",working_electrode_area=2,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[5].electrolyte=lib.Electrolyte(solvent="MeCN",conducting_salt="[Bu$_4$N][PF$_6$]",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[5].analysis=lib.Analysis(cv=lib.CV(scan_rate=20,scan_rate_unit="mV/s",measurement_current_unit="µA",measurement_potential_unit="V"))
e_chem.experiments[6].electrode_setup=lib.ElectrodeSetup(working_electrode="Au",working_electrode_area=2,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[6].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[6].analysis=lib.Analysis(cv=lib.CV(scan_rate=20,scan_rate_unit="mV/s",measurement_current_unit="µA",measurement_potential_unit="V"))
e_chem.experiments[7].electrode_setup=lib.ElectrodeSetup(working_electrode="Au",working_electrode_area=2,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[7].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[7].analysis=lib.Analysis(cv=lib.CV(scan_rate=100,scan_rate_unit="mV/s",measurement_current_unit="µA",measurement_potential_unit="V"))
e_chem.experiments[8].electrode_setup=lib.ElectrodeSetup(working_electrode="Au",working_electrode_area=2,working_electrode_area_unit="cm^2",counter_electrode="Pt",reference_electrode="Ag/AgCl",reference_electrode_salt="KCl",reference_electrode_salt_concentration=1,reference_electrode_salt_concentration_unit="M")
e_chem.experiments[8].electrolyte=lib.Electrolyte(solvent="H$_2$O",conducting_salt="KOH",conducting_salt_concentration=1,conducting_salt_concentration_unit="M",pH=10)
e_chem.experiments[8].analysis=lib.Analysis(cv=lib.CV(scan_rate=100,scan_rate_unit="mV/s",measurement_current_unit="µA",measurement_potential_unit="V"))

In [27]:
e_chem.experiments[0].analysis.cp.potential_end_value

[]

In [8]:

#e_chem.experiments[4].analysis.cv.cycles=[]
print(e_chem.experiments[4].analysis.cv.cycles)

[]


In [10]:
e_chem.experiments[0].type=="CP"

True

# Change the style and parameter of the plots
The following cell is not needed only if you want to change some parameters of your plots

In [4]:
plt.rcParams.update({
    'figure.figsize': (6.4,4.8),     # 4:3 aspect ratio. You can change the figure size here
    'font.size' : 12,                   # Set font size to 12pt
    'axes.labelsize': 12,               # -> axis labels
    'legend.fontsize': 12,              # -> legends
    'font.family': 'serif',             # -> font family  
    'font.sans-serif': 'Times New Roman',
    'text.usetex': False,              # Latex plot rendering. If True,your plots will be rendered with Latex. But this needs extra time for rendering
     'text.latex.preamble':            # LaTeX preamble
         r'\usepackage{mlmodern}'
         #mathptmx = Times
         #... more packages if needed
})
####################
#Here u can also use an other matplotlib style, like ggplot
#plt.style.use('ggplot') # You can choose ure matplotlib style

# If you prefere plotstyle like Origin use the following code:

In [18]:
##### """" For Plotstyle like Origin:
from matplotlib import  cycler
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'Verdana', 'DejaVu Sans']
plt.rcParams['font.size'] = 15
plt.rcParams['axes.linewidth'] = 1.1
plt.rcParams['axes.labelpad'] = 5.0
plot_color_cycle = cycler('color', [ '0000FE', 'FE0000', '008001', 'FD8000', '8c564b', 'e377c2', '7f7f7f', 'bcbd22', '17becf','000000'])
plt.rcParams['axes.prop_cycle'] = plot_color_cycle


# Analytic tools
## ReferenceCalculator class 
The ReferenceCalculator class includes a list of different reference potentials and a reference_difference function. 
- (Reference: https://www.edaq.com/wiki/Reference_Electrode_Potentials (visited on 03.08.23), 
- for ferrocene: Adv. Mater. 2011, 23, 2367–2371, DOI: 10.1002/adma.201004554). 

This can be used to calculate the difference between two reference potentials. The reference_difference function is an interactive calculator.

In [7]:
calculator=ReferenceCalculator(e_chem=e_chem,experiment_list=[0])
calculator.reference_list()
calculator.reference_difference()

Reference                         Potential vs. (NHE) (V)
------------------------------  -------------------------
SHE                                                0
RHE                                               -0.0591
Hg/Hg2Cl2 (0.1 M KCl)                              0.334
Hg/Hg2Cl2 (1 M KCl)                                0.28
Hg/Hg2Cl2 (3.5 M KCl)                              0.25
Hg/Hg2Cl2 (sat. KCl)                               0.241
Hg/Hg2SO4 (0.5 M H2SO4)                            0.682
Hg/Hg2SO4 (1 M H2SO4)                              0.674
Hg/Hg2SO4 (sat. K2SO4)                             0.65
Ag/AgCl (0.1 M KCl)                                0.2881
Ag/AgCl (3 M KCl)                                  0.21
Ag/AgCl (3.5 M KCl)                                0.205
Ag/AgCl (sat. KCl)                                 0.199
Ag/Ag2SO4 (0.5 M H2SO4)                            0.72
Ag/Ag2SO4 (1 M H2SO4)                              0.71
Ag/Ag2SO4 (sat. K2SO4)               

interactive(children=(Dropdown(description='old_reference', index=12, options=('SHE', 'RHE', 'Hg/Hg2Cl2 (0.1 M…

# Chronopotentiometry
- Chronopotentiometry(e_chem,experiment_list,add_potential_value=None,new_reference_name=None,change_reference=False)

This class specifically requires a list of Chronopotentiometry experiments as input. For example, you can pass in [0, 1] to indicate using the first and second experiments that you defined earlier. By setting change_reference=True and specifying a value in add_potential_value, you can conveniently adjust your reference and perform calculations in the new reference scale. Additionally, you have the option to customize the label of your y-axis by defining new_reference_name. It's important to note that this functionality only works when change_reference=True.

The CP class has the following functions:
- quick_plot() -> shows all relevant data: $E$ vs time, $I$ vs. time and $E$ of the mess cell vs time. It is only to check if your data looks good or not
- plot() -> return a plot $E$ vs time. 
- end_value()
- end_value_fit()

To determine the last potential, you have two options:

Using end_value(): This function returns the average of the last 75 values of your measurement by default. However, you can also specify a different range, up to the last 200 values, based on your choice. This method is suitable for general measurements.

Using end_value_fit(): This alternative method determines the last potential through a fit function. It can be particularly useful for handling oscillating measurement data or cases where a fit provides a more accurate representation of the last potential.

By utilizing either end_value() or end_value_fit(), you can determine the last potential in your measurement based on your specific needs and the characteristics of your data.

In [6]:
cp=Chronopotentiometry(e_chem=e_chem,experiment_list=[0,1],change_reference_list_index=0,change_reference=False)
# cp.quick_plot()
#cp.plot()
#cp.end_value()
cp.end_value_fit()

interactive(children=(Text(value='$t$ (s)', description='xlabel'), Text(value='$E$ vs. Hg/HgO (V)', descriptio…

# Chronoamperometry
- Chronoamperometry(e_chem,experiment_list,current_density=False)

This class is very similar to the ChronoPotentiometry class; it has the same input parameter experiment_list. However, it has an additional attribute called current_density, which is set to False by default. By setting current_density to True, you will be able to work with the current density instead of the current in your calculations or analysis.

The CA class has the following functions:
- plot() -> return a plot $I$ vs time
- end_value()
- end_value_fit()

In [13]:
ca=Chronoamperometry(e_chem=e_chem,experiment_list=[2],current_density=True)
#ca.plot()
ca.end_value()
# ca.end_value_fit()

interactive(children=(Text(value='$t$ (s)', description='xlabel'), Text(value='$J$ (A/cm$^2$)', description='y…

# CyclicVoltammetry 
- CyclicVoltammetry(e_chem,experiment,cycles,add_potential_value,new_reference_name,current_density=False,change_reference=False)

The CyclicVoltammetry class is for Cyclic Voltammetry. By set cycles=`None` it automatically uses all cycles which are in your datafile.
It has the following methods:
- plot()
- peaks() 
- integration() 
- ferrocene_reference(experiment2) # This method requires a second experiment. This experiment should involve using your sample without the ferrocene measurement, while using the ferrocene experiment as the initialization step.


In [34]:

cv=CyclicVoltammetry(e_chem=e_chem,experiment=4,cycles=[1],change_reference_list_index=None,current_density=False,change_reference=False)
#cv.plot()
cv.peaks()
#cv.integration()
#cv.ferrocene_reference(experiment2=6)
#cv.integration()


interactive(children=(FloatSlider(value=-1.49903, description='E_Min', max=0.299962, min=-1.49903), FloatSlide…

In [35]:
e_chem.experiments[4].analysis.cv.cycles

[Cycle(id='cycle0', number=1, scan_rate=20.0, scan_rate_unit='mV/s', current_vertex=-9.09213, y_unit_vertex='µA', change_reference_potential=0.0, reference_name='Ag/AgCl', potential_vertex=-9.09213, peaks_and_half_potential=[], peak_integration=[]),
 Cycle(id='cycle1', number=2, scan_rate=20.0, scan_rate_unit='mV/s', current_vertex=None, y_unit_vertex=None, change_reference_potential=None, reference_name=None, potential_vertex=None, peaks_and_half_potential=[], peak_integration=[])]

In [21]:
cv=CyclicVoltammetry(e_chem=e_chem,experiment=6,cycles=[1],add_potential_value=-0.360544,change_reference=True,current_density=True)
cv.plot()

interactive(children=(Text(value='$E$ vs. Fc / Fc$^{+}$ (V)', description='xlabel'), Text(value='$J$ (µA / cm$…

In [12]:
cv=CyclicVoltammetry(e_chem=e_chem,experiment=4,cycles=[1],add_potential_value=None,new_reference_name=None,current_density=True,change_reference=False)
#cv.plot()
cv.peaks()

[Cycle(id='cycle0', cycle_number=[], peak_maxima=[], peak_minima=[], half_wave_potential=[])]
[Cycle(id='cycle0', cycle_number=[], peak_maxima=[], peak_minima=[], half_wave_potential=[]), Cycle(id='cycle1', cycle_number=[], peak_maxima=[], peak_minima=[], half_wave_potential=[])]


interactive(children=(FloatSlider(value=-1.49903, description='E_Min', max=0.299962, min=-1.49903), FloatSlide…